```
# Regex tổng hợp
patterns = {
    'person': r'\b(?:Justice|Mr\.|Dr\.|Chief Justice|Hon\.)\s+(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)(?:\s+[A-Z]\.)?(?:,\s*(Jr\.|Sr\.|III|IV))?',
    'case': r'\b[A-Z][a-zA-Z]*\s+v\.\s+[A-Z][a-zA-Z]*(?:,\s*\d+\s+[A-Z]{1,3}\.\s*\d+\s*\(\d{4}\))?',
    'term': r'\b(?:[A-Z][a-z]+(?:\s+[a-z]+)*\s+[Tt]est|[A-Z][a-z]+\s+[Ll]aws)\b',
    'abbreviation': r'\b(?:[A-Z]\.)(?:[A-Z]\.)?'
}
```

Để trả lời chính xác liệu phương pháp trên đã "vét đủ các cases" chưa, cần xem xét tất cả các trường hợp tiềm năng trong đoạn văn và ngữ cảnh tương tự. Dưới đây là phân tích và kiểm tra các trường hợp có thể gặp phải trong đoạn văn mẫu của bạn:

---

### **1. Các dạng cụm phổ biến trong đoạn văn**
#### a) **Tên người**
- Dạng cơ bản: `Justice William Brennan`
- Dạng có viết tắt: `Justice William J. Brennan`
- Dạng có hậu tố: `Justice William J. Brennan, Jr.`
- Các danh xưng khác: `Mr.`, `Dr.`, `Chief Justice`, `Hon.`
- Tên riêng với nhiều từ: `Warren Earl Burger`

#### b) **Tên vụ kiện**
- Dạng chuẩn: `Miller v. California`
- Dạng mở rộng: `Roth v. United States`
- Dạng cổ điển: `Regina v Hicklin`
- Dạng có chú thích trong văn bản: `Miller v. California, 413 U.S. 15 (1973)`

#### c) **Cụm từ đặc biệt không phải tên**
- Quy định pháp lý: `California Penal Code 311.2(a)`
- Quy tắc hoặc tên thuật ngữ: `Hicklin test`, `Comstock laws`
- Tên tác phẩm, sự kiện: `Paris Adult Theatre I v. Slaton`, `Ashcroft v. Free Speech Coalition`

#### d) **Chữ viết tắt**
- Viết tắt dạng đơn: `U.S.`, `v.`
- Viết tắt theo kiểu học thuật hoặc pháp lý: `Roth v. United States, 354 U.S. 476 (1957)`

---

### **2. Những trường hợp có thể chưa xử lý đủ**
#### **a) Tên người với nhiều phần hoặc dạng khác biệt**
Ví dụ:
- `Chief Justice Warren Earl Burger`
  - Regex hiện tại có thể bỏ sót các cụm tên có nhiều từ hoặc danh xưng phức tạp.

#### **b) Tên vụ kiện có số hiệu hoặc nhiều từ**
Ví dụ:
- `Miller v. California, 413 U.S. 15 (1973)`
  - Regex hiện tại không nhận diện phần số hiệu (`413 U.S. 15`) đi kèm.

#### **c) Các danh từ viết hoa không thuộc nhóm tên người hoặc vụ kiện**
Ví dụ:
- `Hicklin test`, `Roth test`
  - Cần bổ sung nếu muốn bảo toàn các thuật ngữ pháp lý.

#### **d) Các cụm viết tắt đặc biệt**
Ví dụ:
- `v.` (trong "Miller v. California") hoặc `U.S.`
  - Regex hiện tại dễ nhầm lẫn `v.` hoặc `U.S.` với dấu chấm kết thúc câu.

---

### **3. Điều chỉnh để vét đủ các cases**
#### **Regex cập nhật cho từng nhóm:**
1. **Tên người (bao gồm danh xưng, tên dài):**
   ```regex
   \b(?:Justice|Mr\.|Dr\.|Chief Justice|Hon\.)\s+(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)(?:\s+[A-Z]\.)?(?:,\s*(Jr\.|Sr\.|III|IV))?
   ```
   - Xử lý được cả danh xưng phức tạp như `Chief Justice Warren Earl Burger`.

2. **Tên vụ kiện (có số hiệu hoặc mô tả):**
   ```regex
   \b[A-Z][a-zA-Z]*\s+v\.\s+[A-Z][a-zA-Z]*(?:,\s*\d+\s+[A-Z]{1,3}\.\s*\d+\s*\(\d{4}\))?
   ```
   - Nhận diện tên vụ kiện kèm số hiệu như `Roth v. United States, 354 U.S. 476 (1957)`.

3. **Các cụm thuật ngữ đặc biệt:**
   ```regex
   \b(?:[A-Z][a-z]+(?:\s+[a-z]+)*\s+[Tt]est|[A-Z][a-z]+\s+[Ll]aws)\b
   ```
   - Ví dụ: `Hicklin test`, `Comstock laws`.

4. **Viết tắt pháp lý hoặc học thuật:**
   ```regex
   \b(?:[A-Z]\.)(?:[A-Z]\.)?
   ```
   - Nhận diện viết tắt như `U.S.` hoặc `v.` trong ngữ cảnh cụ thể.

---

### **4. Phương pháp tổng hợp**
1. Nhận diện tất cả các cụm theo các regex trên.
2. Thay thế tạm thời bằng placeholder để bảo toàn.
3. Thực hiện tách câu dựa trên dấu chấm (`.`).
4. Khôi phục các cụm gốc từ placeholder.

---

### **5. Mã Python hoàn chỉnh**
```python
import re

# Văn bản mẫu
text = """Justice William J. Brennan, Jr., led the charge for protecting "obscenity". 
Miller v. California, 413 U.S. 15 (1973), was a landmark case. Chief Justice Warren Earl Burger pushed for looser definitions."""

# Regex tổng hợp
patterns = {
    'person': r'\b(?:Justice|Mr\.|Dr\.|Chief Justice|Hon\.)\s+(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)(?:\s+[A-Z]\.)?(?:,\s*(Jr\.|Sr\.|III|IV))?',
    'case': r'\b[A-Z][a-zA-Z]*\s+v\.\s+[A-Z][a-zA-Z]*(?:,\s*\d+\s+[A-Z]{1,3}\.\s*\d+\s*\(\d{4}\))?',
    'term': r'\b(?:[A-Z][a-z]+(?:\s+[a-z]+)*\s+[Tt]est|[A-Z][a-z]+\s+[Ll]aws)\b',
    'abbreviation': r'\b(?:[A-Z]\.)(?:[A-Z]\.)?'
}

# Placeholder dictionaries
placeholders = {}

# Thay thế các cụm nhận diện bằng placeholder
for label, pattern in patterns.items():
    matches = re.findall(pattern, text)
    for i, match in enumerate(matches):
        placeholder = f"PLACEHOLDER_{label.upper()}_{i}"
        placeholders[placeholder] = match
        text = text.replace(match, placeholder)

# Tách câu
sentences = re.split(r'\.(?=\s+[A-Z])', text)

# Khôi phục các cụm gốc
final_sentences = [sentence.strip() for sentence in sentences]
for placeholder, original in placeholders.items():
    final_sentences = [sentence.replace(placeholder, original) for sentence in final_sentences]

# Kết quả
for sentence in final_sentences:
    print(sentence)
```

---

### **6. Đảm bảo "vét đủ cases"**
Với các regex và phương pháp trên:
- **Tên người:** Bao gồm tất cả dạng phức tạp như danh xưng dài, viết tắt, hậu tố.
- **Tên vụ kiện:** Được nhận diện cả số hiệu, năm.
- **Thuật ngữ đặc biệt:** Đảm bảo giữ nguyên các cụm quan trọng như "Hicklin test".
- **Viết tắt:** Không nhầm lẫn với dấu câu.

Nếu bạn phát hiện thêm trường hợp chưa được xử lý, hãy chia sẻ để mình bổ sung! 😊

```
# import tiktoken
# import re
# def split_text(
#     text: str, tokenizer: tiktoken.get_encoding("cl100k_base"), max_tokens: int, overlap: int = 0
# ):
#     """
#     Splits the input text into smaller chunks based on the tokenizer and maximum allowed tokens.
    
#     Args:
#         text (str): The text to be split.
#         tokenizer (CustomTokenizer): The tokenizer to be used for splitting the text.
#         max_tokens (int): The maximum allowed tokens.
#         overlap (int, optional): The number of overlapping tokens between chunks. Defaults to 0.
    
#     Returns:
#         List[str]: A list of text chunks.
#     """
#     # Split the text into sentences using multiple delimiters
#     delimiters = [".", "!", "?", "\n"]
#     regex_pattern = "|".join(map(re.escape, delimiters))
#     sentences = re.split(regex_pattern, text)
#     # print(sentences)
#     # Calculate the number of tokens for each sentence
#     n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]
    
#     chunks = []
#     current_chunk = []
#     current_length = 0
    
#     for sentence, token_count in zip(sentences, n_tokens):
#         # If the sentence is empty or consists only of whitespace, skip it
#         if not sentence.strip():
#             continue
        
#         # If the sentence is too long, split it into smaller parts
#         if token_count > max_tokens:
#             sub_sentences = re.split(r"[,;:]", sentence)
            
#             # there is no need to keep empty os only-spaced strings
#             # since spaces will be inserted in the beginning of the full string
#             # and in between the string in the sub_chuk list
#             filtered_sub_sentences = [sub.strip() for sub in sub_sentences if sub.strip() != ""]
#             sub_token_counts = [len(tokenizer.encode(" " + sub_sentence)) for sub_sentence in filtered_sub_sentences]
            
#             sub_chunk = []
#             sub_length = 0
            
#             for sub_sentence, sub_token_count in zip(filtered_sub_sentences, sub_token_counts):
#                 if sub_length + sub_token_count > max_tokens:
                    
#                     # if the phrase does not have sub_sentences, it would create an empty chunk
#                     # this big phrase would be added anyways in the next chunk append
#                     if sub_chunk:
#                         chunks.append(" ".join(sub_chunk))
#                         sub_chunk = sub_chunk[-overlap:] if overlap > 0 else []
#                         sub_length = sum(sub_token_counts[max(0, len(sub_chunk) - overlap):len(sub_chunk)])
                
#                 sub_chunk.append(sub_sentence)
#                 sub_length += sub_token_count
            
#             if sub_chunk:
#                 chunks.append(" ".join(sub_chunk))
        
#         # If adding the sentence to the current chunk exceeds the max tokens, start a new chunk
#         elif current_length + token_count > max_tokens:
#             chunks.append(" ".join(current_chunk))
#             current_chunk = current_chunk[-overlap:] if overlap > 0 else []
#             current_length = sum(n_tokens[max(0, len(current_chunk) - overlap):len(current_chunk)])
#             current_chunk.append(sentence)
#             current_length += token_count
        
#         # Otherwise, add the sentence to the current chunk
#         else:
#             current_chunk.append(sentence)
#             current_length += token_count
    
#     # Add the last chunk if it's not empty
#     if current_chunk:
#         chunks.append(" ".join(current_chunk))
    
#     return chunks

```

```python
import tiktoken
import re

def process_patterns(text, patterns):
    """
    Xử lý các cụm đặc biệt trong văn bản dựa trên các regex patterns, thay thế chúng bằng placeholders.

    Args:
        text (str): Văn bản đầu vào.
        patterns (dict): Từ điển chứa các regex patterns.

    Returns:
        Tuple[str, dict]: Văn bản đã thay thế placeholders và từ điển chứa mapping từ placeholder đến cụm gốc.
    """
    placeholders = {}
    for label, pattern in patterns.items():
        # Tìm tất cả các cụm khớp với pattern
        matches = re.findall(pattern, text)
        for i, match in enumerate(matches):
            # Tạo placeholder để thay thế cụm khớp
            placeholder = f"PLACEHOLDER_{label.upper()}_{i}"
            # Lấy giá trị khớp đầy đủ
            full_match = match if isinstance(match, str) else match[0]
            # Lưu giá trị khớp vào dictionary để khôi phục sau
            placeholders[placeholder] = full_match
            # Thay thế cụm khớp trong văn bản bằng placeholder
            text = text.replace(full_match, placeholder, 1)
    return text, placeholders

    # Định nghĩa các mẫu regex để nhận diện các cụm đặc biệt
    patterns = {
        'person': r'\b(?:Justice|Mr\.|Dr\.|Chief Justice|Hon\.)\s+(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)(?:\s+[A-Z]\.)?(?:,\s*(Jr\.|Sr\.|III|IV))?',
        'case': r'\b[A-Z][a-zA-Z]*\s+v\.\s+[A-Z][a-zA-Z]*(?:,\s*\d+\s+[A-Z]{1,3}\.\s*\d+\s*\(\d{4}\))?',
        'term': r'\b(?:[A-Z][a-z]+(?:\s+[a-z]+)*\s+[Tt]est|[A-Z][a-z]+\s+[Ll]aws)\b',
        'abbreviation': r'\b(?:[A-Z]\.)(?:[A-Z]\.)?',
        'legal_code': r'\b(?:[A-Z][a-z]+)\s+\d+(\.\d+)?(?:\([a-zA-Z0-9]+\))?'
    }


def split_text(
    text: str, tokenizer: tiktoken.get_encoding("cl100k_base"), max_tokens: int, overlap: int = 0
):
    """
    Chia nhỏ văn bản đầu vào thành các đoạn (chunks) dựa trên số lượng tokens tối đa được phép.
    
    Args:
        text (str): Văn bản cần được chia nhỏ.
        tokenizer (CustomTokenizer): Tokenizer dùng để mã hóa văn bản.
        max_tokens (int): Số lượng tokens tối đa trong mỗi đoạn.
        overlap (int, optional): Số câu trùng lặp giữa các đoạn. Mặc định là 0.
    
    Returns:
        List[str]: Danh sách các đoạn văn bản đã được chia nhỏ.
    """
    # Định nghĩa các mẫu regex để nhận diện các cụm đặc biệt
    patterns = {
        'person': r'\b(?:Justice|Mr\.|Dr\.|Chief Justice|Hon\.)\s+(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)(?:\s+[A-Z]\.)?(?:,\s*(Jr\.|Sr\.|III|IV))?',
        'case': r'\b[A-Z][a-zA-Z]*\s+v\.\s+[A-Z][a-zA-Z]*(?:,\s*\d+\s+[A-Z]{1,3}\.\s*\d+\s*\(\d{4}\))?',
        'term': r'\b(?:[A-Z][a-z]+(?:\s+[a-z]+)*\s+[Tt]est|[A-Z][a-z]+\s+[Ll]aws)\b',
        'abbreviation': r'\b(?:[A-Z]\.)(?:[A-Z]\.)?',
        'legal_code': r'\b(?:[A-Z][a-z]+)\s+\d+(\.\d+)?(?:\([a-zA-Z0-9]+\))?'
    }

    # Gọi hàm xử lý patterns để thay thế các cụm đặc biệt bằng placeholders
    text, placeholders = process_patterns(text, patterns)
    
    # Tách văn bản thành các câu dựa trên dấu câu
    sentence_pattern = r'(?<=[.!?])\s+'
    sentences = re.split(sentence_pattern, text)
    
    # Khôi phục lại các placeholders về nội dung gốc trong từng câu
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    for i, sentence in enumerate(sentences):
        for placeholder, original in placeholders.items():
            # Thay thế placeholders bằng nội dung gốc
            sentences[i] = sentences[i].replace(placeholder, original)
    
    # Tính toán số lượng tokens trong từng câu
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]
    
    # Chia văn bản thành các đoạn (chunks) dựa trên số tokens tối đa
    chunks = []
    current_chunk = []
    current_length = 0

    for i, (sentence, token_count) in enumerate(zip(sentences, n_tokens)):
        # Nếu thêm câu này vào sẽ vượt quá số tokens tối đa, tạo chunk mới
        if current_length + token_count > max_tokens:
            chunks.append(" ".join(current_chunk))  # Thêm chunk hiện tại vào danh sách
            # Chỉ giữ lại số câu overlap ở cuối chunk hiện tại cho chunk tiếp theo
            current_chunk = current_chunk[-overlap:] if overlap > 0 else []
            # Tính lại số tokens của phần overlap
            current_length = sum(
                [len(tokenizer.encode(" " + chunk)) for chunk in current_chunk]
            )
        
        # Thêm câu hiện tại vào chunk
        current_chunk.append(sentence)
        current_length += token_count
    
    # Thêm chunk cuối cùng nếu không rỗng
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

# Ví dụ sử dụng
text = """
Justice William J. Brennan, Jr., led the charge for protecting "obscenity". 
Miller v. California, 413 U.S. 15 (1973), was a landmark case. Chief Justice Warren Earl Burger pushed for looser definitions.
California Penal Code 311.2(a) specifies certain legal restrictions.
"""
# Tạo tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")
# Gọi hàm để chia văn bản
chunks = split_text(text, tokenizer, max_tokens=50, overlap=1)

# In kết quả
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

```

```
chunk trước rồi mới KHÔI PHỤC 

    # Khôi phục lại các placeholders về nội dung gốc trong từng câu
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    for i, sentence in enumerate(sentences):
        for placeholder, original in placeholders.items():
            # Thay thế placeholders bằng nội dung gốc
            sentences[i] = sentences[i].replace(placeholder, original)
```

# Tư duy xài thư viện NLP đã có 